In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)
EXTERNAL_DB = DB.EXTERNAL_DB

# Open Database

In [ ]:
db3d = DB.vehicles_db
db3d.load_data()

# plane = db3d.get_vehicle("hermes")

### Select Plane

In [ ]:
from time import sleep

planenames: list[str] = db3d.get_planenames()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}")
sleep(1)

indexes = [1]  # range(0,len(planenames))
print(len(planenames))
plane = db3d.get_vehicle(planenames[indexes[0]])
states = db3d.get_states(planenames[indexes[0]])
state = states['Unstick']
# dynplane = db3d.dyn_planes[planenames[index]]
planenames = [planenames[i] for i in indexes]

# Add XFLR Polars

In [ ]:
from ICARUS.computation.solvers.XFLR5.polars import read_polars_3d
EXTERNAL_DB = DB.EXTERNAL_DB

for name in planenames:
    if name.startswith("XFLR"):
        continue

    if f"XFLR_{name}" not in planenames:
        try:
            import os

            XFLR5PLANEDIR: str = os.path.join(EXTERNAL_DB, f"{name}.txt")
            read_polars_3d(DB, XFLR5PLANEDIR, name)
            print(f"Imported XFLR polar for {name}")
            planenames.append(f"XFLR_{name}")
        except FileNotFoundError:
            print(f"No XFLR polar found for {name}")
            pass

# Airplane Visualization

In [ ]:
plane.visualize()

# Polars

In [ ]:
# db3d.get_polars("hermes")

In [ ]:
from ICARUS.visualization.airplane.db_polars import plot_airplane_polars

DB.vehicles_db.load_data()
solvers = [
     "GenuVP3 Potential",
     "GenuVP3 2D",
    # "GNVP3 ONERA",
    # "GenuVP7 Potential",
    # "GenuVP7 2D",
    #  "LSPT Potential",
    # "LSPT 2D",/'/
    "AVL"
]
axs, fig = plot_airplane_polars(
    planenames,
    solvers,
    plots=[["AoA", "CL"], ["AoA", "CD"], ["AoA", "Cm"]],
    size=(6, 7),
)

In [ ]:
axs = axs.flatten()
polar = DB.foils_db.get_polars('NACA4415')
for reyn in polar.reynolds_nums[::3]:
    df = polar.get_reynolds_subtable(reyn)

    cl = df['CL']
    aoa = df['AoA'] - 2.8
    cd = df['CD']
    cm = df['Cm']

    axs[0].plot(aoa, cl, label=f'NACA4415 {reyn}')
    axs[1].plot(aoa, cd, label=f'NACA4415 {reyn}')
    axs[2].plot(aoa, cm, label=f'NACA4415 {reyn}')

for ax in axs:
    ax.legend()

In [ ]:
plane.airfoils

In [ ]:
from ICARUS.visualization.airplane.cg_investigation import cg_investigation

cg_investigation(
    planenames[0],
    solvers,
    size=(10, 7),
)

# Case Convergence

In [ ]:
from ICARUS.visualization.airplane.gnvp_convergence import plot_convergence

In [ ]:
angles2show = [0.0]  # All
# Only Works for unsteady solvers: LSPT, GNVP3, GNVP7
plot_convergence(
    plane.name, angles=angles2show, solvers=["All"], plot_error=False, size=(16, 7)
)

# Wake Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_wake import plot_gnvp_wake
from ICARUS.database.utils import angle_to_case

In [ ]:
CASE: str = angle_to_case(-5.0)
plane = db3d.planes[plane.name]
plot_gnvp_wake(3, plane, CASE, scale=True)

 # Loads Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_strips import gnvp_strips_2d, gnvp_strips_3d
CASE: str = angle_to_case(-5.0)

NBs = [i + 1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ", NBs)

stripData = gnvp_strips_3d(plane, CASE, [NBs[0]],genuvp_version= 3, category="Wind")

# Dynamics

In [ ]:
states = db3d.states["bmark"]
for i, name in enumerate(states):
    print(f"{i}: {name}")

In [ ]:
db3d.states["bmark"]

In [ ]:
from ICARUS.flight_dynamics.state import State

states: State = states["plane_1"]
states.stability_fd()
print(states)
states.plot_eigenvalues()

In [ ]:
states.state_space